In [2]:
import numpy as np
from scipy.special import erfc
import random

In [ ]:
class BabyAES:
    S_BOX = {
        '0': 'a', '1': '4', '2': '3', '3': 'b',
        '4': '8', '5': 'e', '6': '2', '7': 'c',
        '8': '5', '9': '7', 'a': '6', 'b': 'f',
        'c': '0', 'd': '1', 'e': '9', 'f': 'd'
    }

    MIX_COLUMNS_MATRIX = np.array([
        [1, 0, 1, 0, 0, 0, 1, 1],
        [1, 1, 0, 1, 0, 0, 0, 1],
        [1, 1, 1, 0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0, 1, 1, 1],
        [0, 0, 1, 1, 1, 0, 1, 0],
        [0, 0, 0, 1, 1, 1, 0, 1],
        [1, 0, 0, 0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0, 1, 0, 1]
    ])


    def __init__(self, key):
        self.key = self.split_key_into_nibbles(key)  
        self.round_keys = self.generate_round_keys()

    def split_key_into_nibbles(self, key):
        return [int(x, 16) for x in f"{key:04x}"]  

    
    def substitute(self, state):
        return [int(self.S_BOX[hex(s)[2:]], 16) for s in state]

    def reverse(self,pair):
        return [pair[1], pair[0]]
    
    def generate_yi(self, i):
        power = (2**(i-1)) & 0xF  
        return [power, 0]

    def generate_round_keys(self):
        w = [
            [self.key[0], self.key[1]],  # w0
            [self.key[2], self.key[3]]   # w1
        ]

        for i in range(1, 5):  # Generate w2, w3, w4, w5, w6, w7, w8, w9
            w2i = [w[2*i - 2][j] ^ self.substitute([self.reverse(w[2*i - 1])[j]])[0] ^ self.generate_yi(i)[j]
                for j in range(2)]
            w2i1 = [w[2*i - 1][j] ^ w2i[j] for j in range(2)]
            w.append(w2i)
            w.append(w2i1)

        round_keys = [w[i] + w[i+1] for i in range(0, 10, 2)]
        # print(round_keys)
        return round_keys

    def mix_columns(self, state):
        binary_matrix = np.zeros((8, 2), dtype=int)  
        for row in range(2):  
            for col in range(2):  
                nibble = state[row][col]  
                binary = format(nibble, '04b')  
                for bit in range(4):  
                    binary_matrix[bit + row * 4][col] = int(binary[bit])  
        mixed = np.dot(self.MIX_COLUMNS_MATRIX, binary_matrix) % 2  
        return mixed

    

    def shift_rows(self, matrix):
        return [[matrix[0][0], matrix[1][0]], [matrix[1][1], matrix[0][1]]]

    def add_round_key(self, state, round_key):
        return [s ^ r for s, r in zip(state, round_key)]
    
    

    def binary_to_hex_list(self, binary_list):
        bList=[]
        for i in range(len(binary_list)):
            bList.append(int("".join(map(str, list(binary_list[i]))), 2))

        return [int(hex(num), 16) for num in bList]

    def encrypt(self, plaintext):
        state = plaintext
        state = self.add_round_key(state, self.round_keys[0])

        for i in range(3):
            state = self.substitute(state)
            
            matrix = [[state[0], state[2]], [state[1], state[3]]]
            matrix = self.shift_rows(matrix)
            
            matrix = self.mix_columns(matrix)
            state_after_mix=[]
            state_after_mix.append(matrix[:4, 0])
            state_after_mix.append(matrix[4:8, 0])
            state_after_mix.append(matrix[4:8, 1])
            state_after_mix.append(matrix[:4, 1])
            state = self.binary_to_hex_list(state_after_mix)
            
            state = self.add_round_key(state, self.round_keys[i+1])
           

        # Final round (no mix_columns)
        state = self.substitute(state)
        
        matrix = [[state[0], state[2]], [state[1], state[3]]]
        matrix = self.shift_rows(matrix)
        
        state = [matrix[0][0], matrix[1][0], matrix[1][1], matrix[0][1]]
        
        state = self.add_round_key(state, self.round_keys[4])
        

        return state
    
    def encrypt_cbc(self, plaintext_blocks, iv):

        ciphertext_blocks = []
        prev_ciphertext = [(iv >> i) & 0xF for i in range(12, -1, -4)]  # Convert IV to nibbles

        for plaintext in plaintext_blocks:
            # XOR plaintext with previous ciphertext (or IV for first block)
            xor_input = [p ^ c for p, c in zip(plaintext, prev_ciphertext)]
            ciphertext = self.encrypt(xor_input)
            ciphertext_blocks.append(ciphertext)

            prev_ciphertext = ciphertext  # Update for next block

        return ciphertext_blocks




In [22]:
key = 0x6b5d
plaintext = [0x2, 0xc, 0xa, 0x5]

baby_aes = BabyAES(key)
ciphertext = baby_aes.encrypt(plaintext)

print("Plaintext:", [hex(x) for x in plaintext])
print("Key:", [hex(x) for x in baby_aes.key])
print("Ciphertext:", [hex(x) for x in ciphertext])

Plaintext: ['0x2', '0xc', '0xa', '0x5']
Key: ['0x6', '0xb', '0x5', '0xd']
Ciphertext: ['0x6', '0x8', '0x5', '0x5']


In [23]:
def completeness_test(plaintext, key):

    baby_aes = BabyAES(key)  
    original_ciphertext = baby_aes.encrypt(plaintext)

    total_bits = len(original_ciphertext) * 4 
    total_flips = 0  
    total_changed_bits = 0 

    for i in range(len(plaintext)):  
        for j in range(4):  
            flipped_plaintext = list(plaintext)  
            flipped_plaintext[i] ^= (1 << j)  

            new_ciphertext = baby_aes.encrypt(flipped_plaintext)

            changed_bits = sum(bin(c1 ^ c2).count('1') for c1, c2 in zip(original_ciphertext, new_ciphertext))
            total_changed_bits += changed_bits
            total_flips += 1

    # Calculate the average percentage of ciphertext bits affected
    avg_percentage = (total_changed_bits / (total_flips * total_bits)) * 100 if total_flips > 0 else 0
    
    return avg_percentage



completeness = completeness_test(plaintext, key)
print(f"Completeness Test Result: {completeness:.2f}%")

Completeness Test Result: 48.44%


In [24]:
def hamming_distance(a, b):
    return bin(a ^ b).count('1')

def avalanche_test(plaintext, key):
    baby_aes = BabyAES(key)
    reference_ciphertext = baby_aes.encrypt(plaintext)  

    total_flips = 0
    total_changed_bits = 0

    for i in range(len(plaintext)):
        for j in range(4):  
            flipped_plaintext = plaintext.copy()
            flipped_plaintext[i] ^= (1 << j)  

            new_ciphertext = baby_aes.encrypt(flipped_plaintext)  
            changed_bits = sum(hamming_distance(c1, c2) for c1, c2 in zip(reference_ciphertext, new_ciphertext))

            print(f"Flipping bit {j} of plaintext[{i}] changed ciphertext:")
            for index, (c1, c2) in enumerate(zip(reference_ciphertext, new_ciphertext)):
                print(f"  Ciphertext Byte {index}: {bin(c1)[2:].zfill(4)} -> {bin(c2)[2:].zfill(4)}")

            total_changed_bits += changed_bits
            total_flips += 1

    return (total_changed_bits / total_flips)

avg_bits_changed = avalanche_test(plaintext, key)
print(f"Avalanche Effect: {avg_bits_changed}average bits changed")


Flipping bit 0 of plaintext[0] changed ciphertext:
  Ciphertext Byte 0: 0110 -> 0101
  Ciphertext Byte 1: 1000 -> 0011
  Ciphertext Byte 2: 0101 -> 0011
  Ciphertext Byte 3: 0101 -> 0101
Flipping bit 1 of plaintext[0] changed ciphertext:
  Ciphertext Byte 0: 0110 -> 0000
  Ciphertext Byte 1: 1000 -> 1010
  Ciphertext Byte 2: 0101 -> 1011
  Ciphertext Byte 3: 0101 -> 0101
Flipping bit 2 of plaintext[0] changed ciphertext:
  Ciphertext Byte 0: 0110 -> 1010
  Ciphertext Byte 1: 1000 -> 0101
  Ciphertext Byte 2: 0101 -> 1001
  Ciphertext Byte 3: 0101 -> 0110
Flipping bit 3 of plaintext[0] changed ciphertext:
  Ciphertext Byte 0: 0110 -> 0010
  Ciphertext Byte 1: 1000 -> 1111
  Ciphertext Byte 2: 0101 -> 0011
  Ciphertext Byte 3: 0101 -> 0110
Flipping bit 0 of plaintext[1] changed ciphertext:
  Ciphertext Byte 0: 0110 -> 1111
  Ciphertext Byte 1: 1000 -> 1110
  Ciphertext Byte 2: 0101 -> 0011
  Ciphertext Byte 3: 0101 -> 0011
Flipping bit 1 of plaintext[1] changed ciphertext:
  Ciphertext B

In [ ]:
def strict_avalanche_test(plaintext, key):
    baby_aes = BabyAES(key)
    reference_ciphertext = baby_aes.encrypt(plaintext)
    
    ciphertext_bit_length = len(reference_ciphertext) * 4  
    plaintext_bit_length = len(plaintext) * 4  

    bit_flip_counts = [0] * ciphertext_bit_length  
    total_flips = plaintext_bit_length  

    for i in range(len(plaintext)):
        for j in range(4):  
            flipped_plaintext = plaintext.copy()
            flipped_plaintext[i] ^= (1 << j)  

            new_ciphertext = baby_aes.encrypt(flipped_plaintext)

            for byte_idx in range(len(reference_ciphertext)):
                for bit_idx in range(4):  
                    ref_bit = (reference_ciphertext[byte_idx] >> bit_idx) & 1
                    new_bit = (new_ciphertext[byte_idx] >> bit_idx) & 1
                    if ref_bit != new_bit:  
                        bit_flip_counts[byte_idx * 4 + bit_idx] += 1  

    flip_probabilities = [count / total_flips for count in bit_flip_counts]

    
    print("Strict Avalanche Test Results:")
    for idx, prob in enumerate(flip_probabilities):
        print(f"Ciphertext bit {idx}: {prob:.2%} flip probability")

    sac_satisfied = all(48 <= (prob * 100) <= 52 for prob in flip_probabilities)  
    return sac_satisfied, flip_probabilities

sac_result, flip_probs = strict_avalanche_test(plaintext, key)
print(f"SAC Satisfied: {sac_result}")


Strict Avalanche Test Results:
Ciphertext bit 0: 31.25% flip probability
Ciphertext bit 1: 50.00% flip probability
Ciphertext bit 2: 56.25% flip probability
Ciphertext bit 3: 31.25% flip probability
Ciphertext bit 4: 62.50% flip probability
Ciphertext bit 5: 62.50% flip probability
Ciphertext bit 6: 56.25% flip probability
Ciphertext bit 7: 43.75% flip probability
Ciphertext bit 8: 62.50% flip probability
Ciphertext bit 9: 56.25% flip probability
Ciphertext bit 10: 62.50% flip probability
Ciphertext bit 11: 43.75% flip probability
Ciphertext bit 12: 37.50% flip probability
Ciphertext bit 13: 50.00% flip probability
Ciphertext bit 14: 37.50% flip probability
Ciphertext bit 15: 31.25% flip probability
SAC Satisfied: False


In [27]:
class DatasetGenerator:
    @staticmethod
    def generate_random_binary(length=16):
        return ''.join(random.choice('01') for _ in range(length))

    @staticmethod
    def generate_avalanche_plaintext(baby_aes, samples=10):
        dataset = []
        for _ in range(samples):
            plaintext = random.getrandbits(16)  
            flipped_plaintext = plaintext ^ (1 << random.randint(0, 15))  

            plaintext_nibbles = [(plaintext >> i) & 0xF for i in range(12, -1, -4)]
            flipped_nibbles = [(flipped_plaintext >> i) & 0xF for i in range(12, -1, -4)]

            ciphertext1 = baby_aes.encrypt(plaintext_nibbles)
            ciphertext2 = baby_aes.encrypt(flipped_nibbles)

            ciphertext1_int = sum(c << (4 * i) for i, c in enumerate(reversed(ciphertext1)))
            ciphertext2_int = sum(c << (4 * i) for i, c in enumerate(reversed(ciphertext2)))

            dataset.append(f"{ciphertext1_int ^ ciphertext2_int:016b}")

        return dataset


    @staticmethod
    def generate_avalanche_key(samples=10):
        dataset = []
        plaintext = random.getrandbits(16)

        plaintext_nibbles = [(plaintext >> i) & 0xF for i in range(12, -1, -4)]

        for _ in range(samples):
            key = random.getrandbits(16)
            flipped_key = key ^ (1 << random.randint(0, 15))

            baby_aes1 = BabyAES(key)
            baby_aes2 = BabyAES(flipped_key)

            ciphertext1 = baby_aes1.encrypt(plaintext_nibbles)
            ciphertext2 = baby_aes2.encrypt(plaintext_nibbles)

            ciphertext1_int = sum(c << (4 * i) for i, c in enumerate(reversed(ciphertext1)))
            ciphertext2_int = sum(c << (4 * i) for i, c in enumerate(reversed(ciphertext2)))

            dataset.append(f"{ciphertext1_int ^ ciphertext2_int:016b}")

        return dataset

    
    @staticmethod
    def generate_plaintext_ciphertext_correlation(baby_aes, samples=10):
        dataset = []
        for _ in range(samples):
            plaintext = random.getrandbits(16)

            plaintext_nibbles = [(plaintext >> i) & 0xF for i in range(12, -1, -4)]
            
            ciphertext = baby_aes.encrypt(plaintext_nibbles)

            ciphertext_int = sum(c << (4 * i) for i, c in enumerate(reversed(ciphertext)))

            dataset.append(f"{plaintext ^ ciphertext_int:016b}")

        return dataset


    @staticmethod
    def generate_cbc_mode(baby_aes, samples=10):
        iv = random.getrandbits(16) 
        plaintext_blocks = [random.getrandbits(16) for _ in range(samples)]
        
        plaintext_nibbles = [[(p >> i) & 0xF for i in range(12, -1, -4)] for p in plaintext_blocks]

        ciphertext_blocks = baby_aes.encrypt_cbc(plaintext_nibbles, iv)

        return [f"{sum(c << (4 * i) for i, c in enumerate(reversed(block))):016b}" for block in ciphertext_blocks]


    @staticmethod
    def generate_random_dataset(samples=10):
        return [DatasetGenerator.generate_random_binary(16) for _ in range(samples)]
    
    @staticmethod
    def generate_low_density(samples=10):
        return ["0" * 15 + "1" for _ in range(samples)]
    
    @staticmethod
    def generate_high_density(samples=10):
        return ["1" * 15 + "0" for _ in range(samples)]

In [28]:

key = 0x6b5d
baby_aes = BabyAES(key)

datasets = {
    "Avalanche Plaintext": DatasetGenerator.generate_avalanche_plaintext(baby_aes),
    "Avalanche Key": DatasetGenerator.generate_avalanche_key(),
    "Plaintext-Ciphertext Correlation": DatasetGenerator.generate_plaintext_ciphertext_correlation(baby_aes),
    "Cipher Block Chaining Mode": DatasetGenerator.generate_cbc_mode(baby_aes),
    "Random": DatasetGenerator.generate_random_dataset(),
    "Low-Density with Plaintext": DatasetGenerator.generate_low_density(),
    "Low-Density with Key": DatasetGenerator.generate_low_density(),
    "High-Density with Plaintext": DatasetGenerator.generate_high_density(),
    "High-Density with Key": DatasetGenerator.generate_high_density(),
}

for name, dataset in datasets.items():
    print(f"{name} Sample: {dataset[0]}")


Avalanche Plaintext Sample: 0101001000111011
Avalanche Key Sample: 0010010100111111
Plaintext-Ciphertext Correlation Sample: 1001010110000010
Cipher Block Chaining Mode Sample: 0001010111010001
Random Sample: 1101011011011011
Low-Density with Plaintext Sample: 0000000000000001
Low-Density with Key Sample: 0000000000000001
High-Density with Plaintext Sample: 1111111111111110
High-Density with Key Sample: 1111111111111110


In [29]:
def monobit_test(dataset):

    total_bits = 0
    ones_count = 0

    for binary_string in dataset:
        ones_count += binary_string.count('1')  
        total_bits += len(binary_string)  

    
    proportion_ones = ones_count / total_bits

    # Check if proportion is within an acceptable range (typically 0.45 to 0.55)
    lower_bound = 0.45  
    upper_bound = 0.55  

    test_passed = lower_bound <= proportion_ones <= upper_bound

    return proportion_ones, test_passed


In [30]:
avalanche_plaintext_dataset = DatasetGenerator.generate_avalanche_plaintext(baby_aes,samples=1000)  

proportion, passed = monobit_test(avalanche_plaintext_dataset)

print(f"Monobit Test - Proportion of 1s: {proportion:.4f}")
print(f"Test Passed: {'Yes' if passed else 'No'}")


Monobit Test - Proportion of 1s: 0.4992
Test Passed: Yes


In [31]:
avalanche_key_dataset = DatasetGenerator.generate_avalanche_key(samples=1000)  

proportion, passed = monobit_test(avalanche_key_dataset)

print(f"Monobit Test - Proportion of 1s: {proportion:.4f}")
print(f"Test Passed: {'Yes' if passed else 'No'}")


Monobit Test - Proportion of 1s: 0.5048
Test Passed: Yes


In [32]:
plaintext_ciphertext_correlation_dataset = DatasetGenerator.generate_plaintext_ciphertext_correlation(baby_aes, samples=1000)  

proportion, passed = monobit_test(plaintext_ciphertext_correlation_dataset)

print(f"Monobit Test - Proportion of 1s: {proportion:.4f}")
print(f"Test Passed: {'Yes' if passed else 'No'}")


Monobit Test - Proportion of 1s: 0.5022
Test Passed: Yes


In [33]:
cbc_mode_dataset = DatasetGenerator.generate_cbc_mode(baby_aes, samples=1000)  

proportion, passed = monobit_test(cbc_mode_dataset)

print(f"Monobit Test - Proportion of 1s: {proportion:.4f}")
print(f"Test Passed: {'Yes' if passed else 'No'}")


Monobit Test - Proportion of 1s: 0.4988
Test Passed: Yes


In [34]:

random_dataset = DatasetGenerator.generate_random_dataset(samples=1000)  

proportion, passed = monobit_test(random_dataset)

print(f"Monobit Test - Proportion of 1s: {proportion:.4f}")
print(f"Test Passed: {'Yes' if passed else 'No'}")


Monobit Test - Proportion of 1s: 0.5026
Test Passed: Yes


In [ ]:
def binary_data_to_hex_list(binary_str):
    if len(binary_str) != 16 or not all(bit in '01' for bit in binary_str):
        raise ValueError("Input must be a 16-bit binary string.")
    chunks = [binary_str[i:i+4] for i in range(0, 16, 4)]
    hex_list = [int(chunk, 2) for chunk in chunks]

    return hex_list

In [ ]:
def hex_list_to_binary(hex_list):
    if len(hex_list) != 4 or not all(isinstance(x, int) for x in hex_list):
        raise ValueError("Input must be a list of 4 hexadecimal numbers (integers).")
    binary_str = ""
    for hex_num in hex_list:
        if hex_num < 0 or hex_num > 0xF:
            raise ValueError("Hexadecimal numbers must be in the range 0x0 to 0xF.")
        
        binary_str += f"{hex_num:04b}"

    return binary_str

In [56]:
low_density_dataset_plaintext = DatasetGenerator.generate_low_density(samples=1000)  
low_density_plaintext_ciphertext = []
monobit_test_low_density_plaintex = []
for i in range(1000):
    low_density_plaintext_ciphertext.append(baby_aes.encrypt(binary_data_to_hex_list(low_density_dataset_plaintext[i])))
    monobit_test_low_density_plaintex.append(hex_list_to_binary(low_density_plaintext_ciphertext[i]))
    
proportion, passed = monobit_test(monobit_test_low_density_plaintex)

print(f"Monobit Test - Proportion of 1s: {proportion:.4f}")
print(f"Test Passed: {'Yes' if passed else 'No'}")


Monobit Test - Proportion of 1s: 0.3125
Test Passed: No


In [57]:
high_density_dataset_plaintext = DatasetGenerator.generate_high_density(samples=1000)  
high_density_plaintext_ciphertext = []
monobit_test_high_density_plaintex = []
for i in range(1000):
    high_density_plaintext_ciphertext.append(baby_aes.encrypt(binary_data_to_hex_list(high_density_dataset_plaintext[i])))
    monobit_test_high_density_plaintex.append(hex_list_to_binary(high_density_plaintext_ciphertext[i]))
    
proportion, passed = monobit_test(monobit_test_high_density_plaintex)

print(f"Monobit Test - Proportion of 1s: {proportion:.4f}")
print(f"Test Passed: {'Yes' if passed else 'No'}")


Monobit Test - Proportion of 1s: 0.5000
Test Passed: Yes


In [ ]:
def binary_to_hex_key(binary_str):
    if len(binary_str) != 16 or not all(bit in '01' for bit in binary_str):
        raise ValueError("Input must be a 16-bit binary string.")

    hex_int = int(binary_str, 2)

    return hex_int

In [73]:
low_density_key = DatasetGenerator.generate_low_density(samples=1000)  
low_density_key_ciphertext = []
monobit_test_low_density_key = []
baby_aes_list = []
for i in range(1000):

    baby_aes_list.append(BabyAES(binary_to_hex_key(low_density_key[i])))
    low_density_key_ciphertext.append(baby_aes_list[i].encrypt(plaintext))
    monobit_test_low_density_key.append(hex_list_to_binary(low_density_key_ciphertext[i]))
    
proportion, passed = monobit_test(monobit_test_low_density_key)

print(f"Monobit Test - Proportion of 1s: {proportion:.4f}")
print(f"Test Passed: {'Yes' if passed else 'No'}")

Monobit Test - Proportion of 1s: 0.8750
Test Passed: No


In [74]:
high_density_key = DatasetGenerator.generate_high_density(samples=1000)  
high_density_key_ciphertext = []
monobit_test_high_density_key = []
baby_aes1_list = []
for i in range(1000):

    baby_aes1_list.append(BabyAES(binary_to_hex_key(high_density_key[i])))
    high_density_key_ciphertext.append(baby_aes1_list[i].encrypt(plaintext))
    monobit_test_high_density_key.append(hex_list_to_binary(high_density_key_ciphertext[i]))
    
proportion, passed = monobit_test(monobit_test_high_density_key)

print(f"Monobit Test - Proportion of 1s: {proportion:.4f}")
print(f"Test Passed: {'Yes' if passed else 'No'}")

Monobit Test - Proportion of 1s: 0.3750
Test Passed: No
